In [3]:
%load_ext autoreload
%autoreload 2

import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd
# modeling은 직접 수정한(position, token embedding을 뺀), 모델로 사용할 것이므로, salt_bert를 활용하면 안됩니다.
import modeling
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
from salt_bert.make_preprocessed_data import tokenization
from salt_bert.make_bert_model import optimization
import random
import logging
import os

random_seed=777
rng = random.Random(random_seed)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(physical_devices[0],'GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

dupe_factor = 100
masked_lm_prob=0.8
max_predictions_per_seq=4
max_seq_length = 7

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import collections
# masked 처리
def create_masked_lm_predictions(tokens, vocab_words, masked_lm_prob=0.15, max_predictions_per_seq=20, rng=rng):
    """Creates the predictions for the masked LM objective."""

    cand_indexes = []
    for (i, token) in enumerate(tokens):			# [MASK] 를 삽입할 token 후보 리스트를 생성
        if token == "[CLS]" or token == "[SEP]":
            continue
        cand_indexes.append(i)

    rng.shuffle(cand_indexes)

    output_tokens = list(tokens)

    masked_lm = collections.namedtuple("masked_lm", ["index", "label"])    # pylint: disable=invalid-name

    num_to_predict = min(max_predictions_per_seq,					# [MASK] 를 삽입할 token의 개수를 현재 token 중에서 15%의 확률로 선택 
                                             max(1, int(round(len(tokens) * masked_lm_prob))))	# 만약 그 값이 20보다 크다면, 20개의 token만 [MASK]로 변환

    masked_lms = []
    covered_indexes = set()
    for index in cand_indexes:
        if len(masked_lms) >= num_to_predict:					# 앞에서 지정한 개수 만큼한 [MASK]
            break
        if index in covered_indexes:
            continue
        covered_indexes.add(index)

        masked_token = None
        # 80% of the time, replace with [MASK]
        if rng.random() < 0.8:							# 그 중, 80%는 [MASK]로 변환
            masked_token = "[MASK]"
        else:
            # 10% of the time, keep original
            if rng.random() < 0.5:							# 나머지 20%의 확률 중, 절반인 10%는 원본 유지
                masked_token = tokens[index]
            # 10% of the time, replace with random word
            else:									# 나머지 10%는 랜덤으로 replace
                masked_token = vocab_words[rng.randint(0, len(vocab_words) - 1)]

        output_tokens[index] = masked_token

        masked_lms.append(masked_lm(index=index, label=tokens[index]))

    masked_lms = sorted(masked_lms, key=lambda x: x.index)

    masked_lm_positions = []
    masked_lm_labels = []
    for p in masked_lms:
        masked_lm_positions.append(p.index)
        masked_lm_labels.append(p.label)

    return (output_tokens, masked_lm_positions, masked_lm_labels)

In [3]:
# 로그 생성
logger = logging.getLogger()

# 로그의 출력 기준 설정
logger.setLevel(logging.INFO)

# log 출력 형식
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# log 출력
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# log를 파일에 출력
# file_handler = logging.FileHandler('my.log')
# file_handler.setFormatter(formatter)
# logger.addHandler(file_handler)


target_strt_date_dt='2020-01-01'
target_strt_str_dt=target_strt_date_dt.replace('-','')
target_end_date_dt='2022-01-01'
target_end_str_dt=target_end_date_dt.replace('-','')



In [4]:
import pandas as pd

# 데이터는 직접 불러와주세요

main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   str_ymd   731 non-null    object
 1   year      731 non-null    int64 
 2   mon       731 non-null    int64 
 3   day       731 non-null    int64 
 4   day_kor   731 non-null    object
 5   weekend   731 non-null    object
 6   holiday   731 non-null    object
 7   str_duty  731 non-null    object
dtypes: int64(3), object(5)
memory usage: 45.8+ KB


In [5]:
# data 전처리를 진행해 주세요.

In [7]:
class TrainingInstance(object):
    """A single training instance (sentence pair)."""

    def __init__(self, tokens, masked_lm_positions, masked_lm_labels):
        self.tokens = tokens
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels

    def __str__(self):
        s = ""
        s += "tokens: %s\n" % (" ".join(
                [tokenization.printable_text(x) for x in self.tokens]))
        s += "masked_lm_positions: %s\n" % (" ".join(
                [str(x) for x in self.masked_lm_positions]))
        s += "masked_lm_labels: %s\n" % (" ".join(
                [tokenization.printable_text(x) for x in self.masked_lm_labels]))
        s += "\n"
        return s

    def __repr__(self):
        return self.__str__()

    
def create_float_feature(values):
    feature = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
    return feature

def create_int_feature(values):
    feature = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
    return feature
    
def write_instance_to_example_files(instances, tokenizer, max_seq_length, max_predictions_per_seq, output_files):
    """Create TF example files from `TrainingInstance`s."""
    writers = []
    for output_file in output_files:
        writers.append(tf.io.TFRecordWriter(output_file))

    writer_index = 0

    total_written = 0
    for (inst_index, instance) in enumerate(instances):
        input_ids = tokenizer.convert_tokens_to_ids(instance.tokens)
        assert len(input_ids) <= max_seq_length

        while len(input_ids) < max_seq_length:
            input_ids.append(0)

        assert len(input_ids) == max_seq_length

        masked_lm_positions = list(instance.masked_lm_positions)
        masked_lm_ids = tokenizer.convert_tokens_to_ids(instance.masked_lm_labels)
        masked_lm_weights = [1.0] * len(masked_lm_ids)

        while len(masked_lm_positions) < max_predictions_per_seq:
            masked_lm_positions.append(0)
            masked_lm_ids.append(0)
            masked_lm_weights.append(0.0)

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(input_ids)
        features["masked_lm_positions"] = create_int_feature(masked_lm_positions)
        features["masked_lm_ids"] = create_int_feature(masked_lm_ids)
        features["masked_lm_weights"] = create_float_feature(masked_lm_weights)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))

        writers[writer_index].write(tf_example.SerializeToString())
        writer_index = (writer_index + 1) % len(writers)

        total_written += 1

        if inst_index < 20:
            logger.info("*** Example ***")
            logger.info("tokens: %s" % " ".join([tokenization.printable_text(x) for x in instance.tokens]))

            for feature_name in features.keys():
                feature = features[feature_name]
                values = []
                if feature.int64_list.value:
                    values = feature.int64_list.value
                elif feature.float_list.value:
                    values = feature.float_list.value
                logger.info("%s: %s" % (feature_name, " ".join([str(x) for x in values])))

    for writer in writers:
        writer.close()

    logger.info("Wrote %d total instances", total_written)
    
def create_instances_from_document(
        all_documents, document_index, masked_lm_prob, max_predictions_per_seq, vocab_words, rng):
    """Creates `TrainingInstance`s for a single document."""
    document = all_documents[document_index]

    instances = []
    current_chunk = []
    current_length = 0
    i = 0
    
    input_token = []
    # CLS/SEP 제외
    # input_token.append("[CLS]")
    input_token.extend(document)
    # input_token.append("[SEP]")

    (tokens, masked_lm_positions,
     masked_lm_labels) = create_masked_lm_predictions(input_token, vocab_words, masked_lm_prob, max_predictions_per_seq, rng)		# 만들어진 전체 sequence의 특정 token index에 [MASK] 를 수행
    instance = TrainingInstance(
            tokens=input_token,
            masked_lm_positions=masked_lm_positions,
            masked_lm_labels=masked_lm_labels)
    instances.append(instance)

    return instances

In [8]:
train_features = # pre training에 사용될 feature값
# vocab.list는 직접 만들어주셔야 합니다. 데이터는 제공되지 않아요!
tokenizer = tokenization.FullTokenizer(vocab_file='./vocab.list', do_lower_case=False)
vocab_words = list(tokenizer.vocab.keys())

instances = []
for _ in range(dupe_factor):
    for document_index in range(len(train_features)):
        instances.extend(
                create_instances_from_document(
                        train_features, document_index, masked_lm_prob, max_predictions_per_seq, vocab_words, rng))
rng.shuffle(instances)

In [9]:
write_instance_to_example_files(instances, tokenizer, max_seq_length, max_predictions_per_seq, ["./test.tf_record"])

2022-01-19 14:23:12,466 - root - INFO - *** Example ***
2022-01-19 14:23:12,467 - root - INFO - tokens: N O O D D E O
2022-01-19 14:23:12,467 - root - INFO - input_ids: 5 6 6 3 3 4 6
2022-01-19 14:23:12,468 - root - INFO - input_mask: 1 1 1 1 1 1 1
2022-01-19 14:23:12,468 - root - INFO - masked_lm_positions: 0 3 4 6
2022-01-19 14:23:12,468 - root - INFO - masked_lm_ids: 5 3 3 6
2022-01-19 14:23:12,469 - root - INFO - masked_lm_weights: 1.0 1.0 1.0 1.0
2022-01-19 14:23:12,470 - root - INFO - *** Example ***
2022-01-19 14:23:12,470 - root - INFO - tokens: E O N N O O D
2022-01-19 14:23:12,470 - root - INFO - input_ids: 4 6 5 5 6 6 3
2022-01-19 14:23:12,470 - root - INFO - input_mask: 1 1 1 1 1 1 1
2022-01-19 14:23:12,471 - root - INFO - masked_lm_positions: 1 3 4 5
2022-01-19 14:23:12,471 - root - INFO - masked_lm_ids: 6 5 6 6
2022-01-19 14:23:12,471 - root - INFO - masked_lm_weights: 1.0 1.0 1.0 1.0
2022-01-19 14:23:12,472 - root - INFO - *** Example ***
2022-01-19 14:23:12,472 - root -

In [10]:
# BERT Config는 적절히 수정하셔서 사용하셔야됩니다. (vocab_size같은거)
bert_config_file = './config/bert_config.json'
do_train = True
do_eval = True
input_file = './test.tf_record'
output_dir = './model_output'
save_checkpoints_steps = 10
learning_rate = 5e-5
train_batch_size = 14
eval_batch_size = 14
num_train_steps = 100
num_warmup_steps= -1
max_eval_steps=20

In [11]:
def gather_indexes(sequence_tensor, positions):
    """Gathers the vectors at the specific positions over a minibatch."""
    sequence_shape = modeling.get_shape_list(sequence_tensor, expected_rank=3)
    batch_size = sequence_shape[0]
    seq_length = sequence_shape[1]
    width = sequence_shape[2]

    flat_offsets = tf.reshape(
        tf.range(0, batch_size, dtype=tf.int32) * seq_length, [-1, 1])
    flat_positions = tf.reshape(positions + flat_offsets, [-1])
    flat_sequence_tensor = tf.reshape(sequence_tensor,
                                      [batch_size * seq_length, width])
    output_tensor = tf.gather(flat_sequence_tensor, flat_positions)
    return output_tensor

In [12]:
def get_masked_lm_output(bert_config, input_tensor, output_weights, positions,
                         label_ids, label_weights):
    """Get loss and log probs for the masked LM."""
    # 텐서 값의 특정 포지션을 가져온다. 여기서는 masking 처리된 값만 가져옴.
    input_tensor = gather_indexes(input_tensor, positions)

    with tf.variable_scope("cls/predictions"):
        with tf.variable_scope("transform"):
            input_tensor = tf.layers.dense(
                input_tensor,
                units=bert_config.hidden_size,
                activation=modeling.get_activation(bert_config.hidden_act),
                kernel_initializer=modeling.create_initializer(
                    bert_config.initializer_range))
            input_tensor = modeling.layer_norm(input_tensor)

        # The output weights are the same as the input embeddings, but there is
        # an output-only bias for each token.
        output_bias = tf.get_variable(
            "output_bias",
            shape=[bert_config.vocab_size],
            initializer=tf.zeros_initializer())

        logits = tf.matmul(input_tensor, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        label_ids = tf.reshape(label_ids, [-1])
        label_weights = tf.reshape(label_weights, [-1])

        one_hot_labels = tf.one_hot(
            label_ids, depth=bert_config.vocab_size, dtype=tf.float32)

        per_example_loss = - \
            tf.reduce_sum(log_probs * one_hot_labels, axis=[-1])
        numerator = tf.reduce_sum(label_weights * per_example_loss)
        denominator = tf.reduce_sum(label_weights) + 1e-5
        loss = numerator / denominator

    return (loss, per_example_loss, log_probs)

In [13]:
def model_fn_builder(bert_config, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        logging.info("*** Features ***")
        for name in sorted(features.keys()):
            logging.info("  name = %s, shape = %s" %
                            (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        masked_lm_positions = features["masked_lm_positions"]
        masked_lm_ids = features["masked_lm_ids"]
        masked_lm_weights = features["masked_lm_weights"]

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        model = modeling.BertModel(
            config=bert_config,
            is_training=is_training,
            input_ids=input_ids,
            input_mask=input_mask,
            use_one_hot_embeddings=use_one_hot_embeddings)

        (masked_lm_loss,
         masked_lm_example_loss, masked_lm_log_probs) = get_masked_lm_output(
             bert_config, model.get_sequence_output(), model.get_embedding_table(),
             masked_lm_positions, masked_lm_ids, masked_lm_weights)

        # 2개의 예측에 대한 토탈 로스
        total_loss = masked_lm_loss

        tvars = tf.trainable_variables()

        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(
                        init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                            init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            # 토탈 로스를 최소화 하기위한 옵티마이저
            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold_fn=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:
            # 검증에서는 비교적 상세 정보를 출력해주도록 함.
            def metric_fn(masked_lm_example_loss, masked_lm_log_probs, masked_lm_ids,
                          masked_lm_weights):
                """Computes the loss and accuracy of the model."""
                masked_lm_log_probs = tf.reshape(masked_lm_log_probs,
                                                 [-1, masked_lm_log_probs.shape[-1]])
                masked_lm_predictions = tf.argmax(
                    masked_lm_log_probs, axis=-1, output_type=tf.int32)
                masked_lm_example_loss = tf.reshape(
                    masked_lm_example_loss, [-1])
                masked_lm_ids = tf.reshape(masked_lm_ids, [-1])
                masked_lm_weights = tf.reshape(masked_lm_weights, [-1])
                masked_lm_accuracy = tf.metrics.accuracy(
                    labels=masked_lm_ids,
                    predictions=masked_lm_predictions,
                    weights=masked_lm_weights)
                masked_lm_mean_loss = tf.metrics.mean(
                    values=masked_lm_example_loss, weights=masked_lm_weights)

                return {
                    "masked_lm_accuracy": masked_lm_accuracy,
                    "masked_lm_loss": masked_lm_mean_loss
                }

            eval_metrics = (metric_fn, [
                masked_lm_example_loss, masked_lm_log_probs, masked_lm_ids,
                masked_lm_weights
            ])
            output_spec = tf.estimator.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metrics=eval_metrics,
                scaffold_fn=scaffold_fn)
        else:
            raise ValueError(
                "Only TRAIN and EVAL modes are supported: %s" % (mode))

        return output_spec

    return model_fn

In [14]:
def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.io.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
            t = tf.cast(t,tf.int32)
        example[name] = t

    return example

In [15]:
def input_fn_builder(input_files,
                     max_seq_length,
                     max_predictions_per_seq,
                     is_training,
                     num_cpu_threads=4):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""
    
    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        name_to_features = {
            "input_ids":
                tf.io.FixedLenFeature([max_seq_length], tf.int64),
            # NSP를 하지 않으므로, 값이 살짝 다릅니다 참고하세요
            "input_mask":
                tf.io.FixedLenFeature([max_seq_length], tf.int64),
            "masked_lm_positions":
                tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
            "masked_lm_ids":
                tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
            "masked_lm_weights":
                tf.io.FixedLenFeature([max_predictions_per_seq], tf.float32)
        }

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        if is_training:
            d = tf.data.Dataset.from_tensor_slices(tf.constant(input_files))
            d = d.repeat()
            d = d.shuffle(buffer_size=len(input_files))

            # `cycle_length` is the number of parallel files that get read.
            cycle_length = min(num_cpu_threads, len(input_files))

            # `sloppy` mode means that the interleaving is not exact. This adds
            # even more randomness to the training pipeline.
            d = d.apply(
                tf.data.experimental.parallel_interleave(
                    tf.data.TFRecordDataset,
                    sloppy=is_training,
                    cycle_length=cycle_length))
            d = d.shuffle(buffer_size=100)
        else:
            d = tf.data.TFRecordDataset(input_files)
            # Since we evaluate for a fixed number of steps we don't want to encounter
            # out-of-range exceptions.
            d = d.repeat()

        # We must `drop_remainder` on training because the TPU requires fixed
        # size dimensions. For eval, we assume we are evaluating on the CPU or GPU
        # and we *don't* want to drop the remainder, otherwise we wont cover
        # every sample.
        d = d.apply(
            tf.data.experimental.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                num_parallel_batches=num_cpu_threads,
                drop_remainder=True))
        return d

    return input_fn

In [16]:
tf.logging.set_verbosity(tf.logging.INFO)

if not do_train and not do_eval:
    raise ValueError("At least one of `do_train` or `do_eval` must be True.")

# TPU로 BERT학습을 시키기 위한 default config 호출
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

tf.gfile.MakeDirs(output_dir)

input_files = []
for input_pattern in input_file.split(","):
    input_files.extend(tf.gfile.Glob(input_pattern))

tf.logging.info("*** Input Files ***")
for input_file in input_files:
    tf.logging.info("input_file_name:  %s" % input_file)

init_checkpoint = None
num_warmup_steps = 10000
use_tpu = False

tpu_cluster_resolver = None
if use_tpu and tpu_name:
    # tpu 활성화
    tpu_cluster_resolver = tf.estimator.tpu.cluster_resolver.TPUClusterResolver(
        tpu_name, zone=tpu_zone, project=gcp_project)

is_per_host = tf.estimator.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.estimator.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=None,
    model_dir=output_dir,
    save_checkpoints_steps=save_checkpoints_steps,
    tpu_config=tf.estimator.tpu.TPUConfig(
        iterations_per_loop=1000,
        num_shards=8,
        per_host_input_for_training=is_per_host))

# 메소드 편하게 쓰기위한 파라미터 세팅
model_fn = model_fn_builder(
    bert_config=bert_config,
    init_checkpoint=init_checkpoint,
    learning_rate=learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=use_tpu,
    use_one_hot_embeddings=use_tpu)

# TPU를 쓸거면 TPU 설정을 하고, TPU가 없다면, 로컬 세팅을 설정하게 됨.
# 학습해야할 모델을 model_fn을 통해 bert로 설정한다. (modeling 클래스의 제공되는 BERT Archi)
estimator = tf.estimator.tpu.TPUEstimator(
    use_tpu=use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=train_batch_size,
    eval_batch_size=eval_batch_size)

if do_train:
    # masked된 여러 샘플들을 계속 배치수만큼 꺼내서 estimator에 선언된 모델로 학습을 진행시킨다.
    tf.logging.info("***** Running training *****")
    tf.logging.info("  Batch size = %d", train_batch_size)
    train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=max_seq_length,
        max_predictions_per_seq=max_predictions_per_seq,
        is_training=True)
    estimator.train(input_fn=train_input_fn,
                    max_steps=num_train_steps)

if do_eval:
    tf.logging.info("***** Running evaluation *****")
    tf.logging.info("  Batch size = %d", eval_batch_size)
    # 순서대로 그냥 만들어놓은 tfrecode값을 가져와서 하나하나 bert모델을 검증하게된다.
    eval_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=max_seq_length,
        max_predictions_per_seq=max_predictions_per_seq,
        is_training=False)
    # 모델 검증
    result = estimator.evaluate(
        input_fn=eval_input_fn, steps=max_eval_steps)
    # 검증결과 저장
    output_eval_file = os.path.join(output_dir, "eval_results.txt")
    # pre-trained 모델 저장
    with tf.gfile.GFile(output_eval_file, "w") as writer:
        tf.logging.info("***** Eval results *****")
        for key in sorted(result.keys()):
            tf.logging.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:*** Input Files ***


2022-01-19 14:23:16,822 - tensorflow - INFO - *** Input Files ***


INFO:tensorflow:input_file_name:  .\test.tf_record


2022-01-19 14:23:16,823 - tensorflow - INFO - input_file_name:  .\test.tf_record


2022-01-19 14:23:16,885 - tensorflow - WARNING - Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x000001AE5EB2BDC0>) includes params argument, but params are not passed to Estimator.


...

2022-01-19 14:25:31,153 - tensorflow - INFO -   loss = 0.0029329616


INFO:tensorflow:  masked_lm_accuracy = 1.0


2022-01-19 14:25:31,154 - tensorflow - INFO -   masked_lm_accuracy = 1.0


INFO:tensorflow:  masked_lm_loss = 0.002932962


2022-01-19 14:25:31,154 - tensorflow - INFO -   masked_lm_loss = 0.002932962
